**Question 3: Transportation Problem**


In [ ]:
!pip -q install pyomo
!apt-get -q install -y -qq coinor-cbc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Selecting previously unselected package coinor-libcoinutils3v5:amd64.
(Reading database ... 120875 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.11.4+repack1-2_amd64.deb ...
Unpacking coinor-libcoinutils3v5:amd64 (2.11.4+repack1-2) ...
Selecting previously unselected package coinor-libosi1v5:amd64.
Preparing to unpack .../1-coinor-libosi1v5_0.108.6+repack1-2_amd64.deb ...
Unpacking coinor-libosi1v5:amd64 (0.108.6+repack1-2) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.17.5+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.17.5+repack1-1) ...
Selecting previously unselected package coinor-libcgl1:amd64.
Preparing to unpack .../3-coinor-libcgl1_0.60.3+repack1-3_amd64.deb ...
Unpacking coinor-libcgl1:amd64 (0.60.3+repack1-3

In [ ]:
# number of variable representing warehouse (M=8) and markets (N=8)
M, N = 8,8

# since variable are two dimensional ie., x_{ij}, therefore both row and column indicies are defined below
row_indicies = np.arange(M)
col_indicies = np.arange(N)

# matrix of cost cofficient
cost = np.array([[627, 617, 1270, 2072, 1415,2943, 1064, 4265], [159, 1432, 351, 3372, 237, 3373, 1456, 5059], [609, 1943, 77, 3365, 298, 3021, 1484, 4803], [2143, 671, 3876, 537, 1902, 1941, 1682, 3348],
                 [2742, 2089, 1931, 2199, 1226, 739, 691, 1128],[2053, 860, 2995, 68, 2069, 2065, 993, 3342],[656, 817, 1245, 2141, 1534, 3351, 1137, 3887],[195, 1862, 1331, 2921, 402, 2825, 563, 4441]])

warehouse_cap = np.array([185,350,405,280,105,75,425,270])
market_demand = np.array([175, 400, 550, 70, 650, 100, 65, 85])
# defining an empty model using pyomo
TransModel = ConcreteModel()

# defining varibles used in model x_{ij} (in total 8*8 = 64 variables)
TransModel.x = Var(row_indicies, col_indicies, domain= NonNegativeReals)

# setting up the objective function
TransModel.Objective = Objective(expr = sum(cost[i][j]*TransModel.x[i,j] for i in row_indicies for j in col_indicies), sense = minimize)

In [ ]:
# below we define constraints based on available demand at hub and capacity at plant
# creating empty list of constraints
TransModel.constraints = ConstraintList()

# adding constraint for production capacity
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for j in col_indicies) <= warehouse_cap[i]) for i in row_indicies]
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for i in row_indicies) == market_demand[j]) for j in col_indicies]

# printing out pyomo model
TransModel.pprint()

4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [ ]:
# installing cbc solver in collab session
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(TransModel)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
import pandas as pd
result1 = pd.DataFrame({'M1':[TransModel.x[i,0].value for i in row_indicies],
                        'M2':[TransModel.x[i,1].value for i in row_indicies],
                        'M3':[TransModel.x[i,2].value for i in row_indicies],
                        'M4':[TransModel.x[i,3].value for i in row_indicies],
                        'M5':[TransModel.x[i,4].value for i in row_indicies],
                        'M6':[TransModel.x[i,5].value for i in row_indicies],
                        'M7':[TransModel.x[i,6].value for i in row_indicies],
                        'M8':[TransModel.x[i,7].value for i in row_indicies]})

In [ ]:
print(result1)


      M1     M2     M3    M4     M5    M6    M7    M8
0    0.0  185.0    0.0   0.0    0.0   0.0   0.0   0.0
1    0.0    0.0    0.0   0.0  350.0   0.0   0.0   0.0
2    0.0    0.0  375.0   0.0   30.0   0.0   0.0   0.0
3    0.0  200.0    0.0   0.0    0.0  80.0   0.0   0.0
4    0.0    0.0    0.0   0.0    0.0  20.0   0.0  85.0
5    0.0    0.0    0.0  70.0    0.0   0.0   5.0   0.0
6  175.0   15.0  175.0   0.0    0.0   0.0  60.0   0.0
7    0.0    0.0    0.0   0.0  270.0   0.0   0.0   0.0


In [ ]:
TransModel.Objective()

1166465.0

In [ ]:
#3.3
# number of variable representing warehouse (M=8) and markets (N=8)
M, N = 8,8

# since variable are two dimensional ie., x_{ij}, therefore both row and column indicies are defined below
row_indicies = np.arange(M)
col_indicies = np.arange(N)

# matrix of cost cofficient
cost = np.array([[627, 617, 1270, 2072, 1415,2943, 1064, 4265], [100000, 1432, 351, 3372, 237, 3373, 1456, 5059], [609, 1943, 77, 3365, 298, 3021, 1484, 4803], [2143, 671, 3876, 537, 1902, 1941, 1682, 3348],
                 [2742, 2089, 1931, 2199, 1226, 739, 691, 100000],[2053, 860, 2995, 100000, 2069, 2065, 993, 3342],[656, 817, 1245, 2141, 1534, 3351, 1137, 3887],[195, 1862, 1331, 2921, 402, 2825, 563, 4441]])

warehouse_cap = np.array([185,350,405,280,105,75,425,270])
market_demand = np.array([175, 400, 550, 70, 650, 100, 65, 85])
# defining an empty model using pyomo
TransModel = ConcreteModel()

# defining varibles used in model x_{ij} (in total 8*8 = 64 variables)
TransModel.x = Var(row_indicies, col_indicies, domain= NonNegativeReals)

# setting up the objective function
TransModel.Objective = Objective(expr = sum(cost[i][j]*TransModel.x[i,j] for i in row_indicies for j in col_indicies), sense = minimize)

In [ ]:
# below we define constraints based on available demand at hub and capacity at plant
# creating empty list of constraints
TransModel.constraints = ConstraintList()

# adding constraint for production capacity
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for j in col_indicies) <= warehouse_cap[i]) for i in row_indicies]
[TransModel.constraints.add(expr = sum(TransModel.x[i,j] for i in row_indicies) == market_demand[j]) for j in col_indicies]

# printing out pyomo model
TransModel.pprint()


4 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   16 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   64 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
       

In [ ]:
8# installing cbc solver in collab session
!apt-get install -y -qq coinor-cbc
# using solverfactory to solve above constructed pyomo model
optCBC = SolverFactory('cbc')
result = optCBC.solve(TransModel)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


In [ ]:
import pandas as pd
result2 = pd.DataFrame({'M1':[TransModel.x[i,0].value for i in row_indicies],
                        'M2':[TransModel.x[i,1].value for i in row_indicies],
                        'M3':[TransModel.x[i,2].value for i in row_indicies],
                        'M4':[TransModel.x[i,3].value for i in row_indicies],
                        'M5':[TransModel.x[i,4].value for i in row_indicies],
                        'M6':[TransModel.x[i,5].value for i in row_indicies],
                        'M7':[TransModel.x[i,6].value for i in row_indicies],
                        'M8':[TransModel.x[i,7].value for i in row_indicies]})

In [ ]:
print(result2)

      M1     M2     M3    M4     M5     M6    M7    M8
0    0.0  185.0    0.0   0.0    0.0    0.0   0.0   0.0
1    0.0    0.0    0.0   0.0  350.0    0.0   0.0   0.0
2    0.0    0.0  375.0   0.0   30.0    0.0   0.0   0.0
3    0.0  200.0    0.0  70.0    0.0    0.0   0.0  10.0
4    0.0    0.0    0.0   0.0    0.0  100.0   5.0   0.0
5    0.0    0.0    0.0   0.0    0.0    0.0   0.0  75.0
6  175.0   15.0  175.0   0.0    0.0    0.0  60.0   0.0
7    0.0    0.0    0.0   0.0  270.0    0.0   0.0   0.0


In [ ]:
TransModel.Objective()

1289875.0